<center>
<img src="https://habrastorage.org/files/fd4/502/43d/fd450243dd604b81b9713213a247aa20.jpg">
## [mlcourse.ai](https://mlcourse.ai) – Open Machine Learning Course 
<center>Author: [Mariya Mansurova](https://www.linkedin.com/in/mariya-mansurova-04070982/), analyst & developer in Yandex.Metrics team. <br>Translated by [Ivan Zakharov](https://www.linkedin.com/in/ivan-zakharov/), ML enthusiast.
<br>All content is distributed under the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license.

# <center> Assignment #9 (demo)
## <center> Time series analysis

**Fill cells marked with "Your code here" and submit your answers to the questions through the [web form](https://docs.google.com/forms/d/1UYQ_WYSpsV3VSlZAzhSN_YXmyjV7YlTP8EYMg8M8SoM/edit).**

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go
import requests
import pandas as pd

print(__version__) # need 1.9.0 or greater
init_notebook_mode(connected = True)


def plotly_df(df, title = ''):
    data = []
    
    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)
    
    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)

4.14.1


## Data preparation

In [2]:
df = pd.read_csv('../data/wiki_machine_learning.csv', sep = ' ')
df = df[df['count'] != 0]
df.head()

,date,count,lang,page,rank,month,title
81,2015-01-01,1414,en,Machine_learning,8708,201501,Machine_learning
80,2015-01-02,1920,en,Machine_learning,8708,201501,Machine_learning
79,2015-01-03,1338,en,Machine_learning,8708,201501,Machine_learning
78,2015-01-04,1404,en,Machine_learning,8708,201501,Machine_learning
77,2015-01-05,2264,en,Machine_learning,8708,201501,Machine_learning


In [3]:
df.shape

(383, 7)

In [4]:
df.describe()

,count,rank,month
count,383.000000,383.0,383.000000
mean,2337.532637,8708.0,201511.462141
std,660.041914,0.0,21.310544
min,726.000000,8708.0,201501.000000
25%,1724.000000,8708.0,201504.000000
50%,2434.000000,8708.0,201507.000000
75%,2785.000000,8708.0,201510.000000
max,4932.000000,8708.0,201601.000000


In [5]:
df['date'].min(),df['date'].max()

('2015-01-01', '2016-01-20')

## Predicting with FB Prophet
We will train at first 5 months and predict the number of trips for June.

In [6]:
df.date = pd.to_datetime(df.date)

In [7]:
plotly_df(df.set_index('date')[['count']])

In [8]:
from fbprophet import Prophet

In [9]:
predictions = 30
df_copy = df.copy()
df = df_copy[['date', 'count']]
df.columns = ['ds', 'y']
df.tail()

,ds,y
382,2016-01-16,1644
381,2016-01-17,1836
376,2016-01-18,2983
375,2016-01-19,3389
372,2016-01-20,3559


In [10]:
model = Prophet()
model.fit(df[:-predictions])
future = model.make_future_dataframe(periods=predictions)
forecast = model.predict(future)
forecast.tail(n=1)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
382,2016-01-20,2992.317986,2999.046376,3840.061273,2966.904472,3016.019582,425.46452,425.46452,425.46452,425.46452,425.46452,425.46452,0.0,0.0,0.0,3417.782506


**<font color='red'>Question 1:</font>** What is the prediction of the number of views of the wiki page on January 20? Round to the nearest integer.

- 4947
- 3426 --
- 5229
- 2744

In [11]:
# You code here
def calculate_forecast_errors(df, prediction_size):
    """Calculate MAPE and MAE of the forecast.
    
       Args:
           df: joined dataset with 'y' and 'yhat' columns.
           prediction_size: number of days at the end to predict.
    """
    
    # Make a copy
    df = df.copy()
    
    # Now we calculate the values of e_i and p_i according to the formulas given in the article above.
    df['e'] = df['y'] - df['yhat']
    df['p'] = 100 * df['e'] / df['y']
    
    # Recall that we held out the values of the last `prediction_size` days
    # in order to predict them and measure the quality of the model. 
    
    # Now cut out the part of the data which we made our prediction for.
    predicted_part = df[-prediction_size:]
    
    # Define the function that averages absolute error values over the predicted part.
    error_mean = lambda error_name: np.mean(np.abs(predicted_part[error_name]))
    
    # Now we can calculate MAPE and MAE and return the resulting dictionary of errors.
    return {'MAPE': error_mean('p'), 'MAE': error_mean('e')}
df['ds'] = pd.to_datetime(df['ds'])
merged = df.set_index('ds').join(forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']])
calculate_forecast_errors(merged, predictions)

{'MAPE': 34.19380159794991, 'MAE': 593.3705390372357}

Estimate the quality of the prediction with the last 30 points.

**<font color='red'>Question 2:</font> What is MAPE equal to?**

- 34.5 -- 
- 42.42
- 5.39
- 65.91

**<font color='red'>Question 3:</font> What is MAE equal to?**

- 355
- 4007
- 600 -- 
- 903

## Predicting with ARIMA

In [12]:
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.api as sm
plt.rcParams['figure.figsize'] = (15, 10)
sm.tsa.stattools.adfuller(df['y'][:-predictions])

(-2.5341178434183047,
 0.10739219627612018,
 15,
 337,
 {'1%': -3.4499043309021955,
  '5%': -2.870154885338925,
  '10%': -2.5713597190254385},
 4878.896813791136)

**<font color='red'>Question 4:</font> Let's verify the stationarity of the series using the Dickey-Fuller test. Is the series stationary? What is the p-value?**

- Series is stationary, p_value = 0.107
- Series is not stationary, p_value = 0.107 --
- Series is stationary, p_value = 0.001
- Series is not stationary, p_value = 0.001

In [13]:
# You code here
from itertools import product                    # some useful functions
from tqdm import tqdm_notebook
# setting initial values and some bounds for them
ps = [1,2]
# d=1 
qs = [1,2]
Ps = [1,3]
# D=1 
Qs = range(0, 2)
s = 7

# creating list with all the possible combinations of parameters
parameters = product( Qs, qs, Ps,ps)
parameters_list = list(parameters)
len(parameters_list)

16

In [14]:
def optimizeSARIMA(parameters_list, d, D, s):
    """
        Return dataframe with parameters and corresponding AIC
        
        parameters_list - list with (p, q, P, Q) tuples
        d - integration order in ARIMA model
        D - seasonal integration order 
        s - length of season
    """
    
    results = []
    best_aic = float("inf")

    for param in tqdm_notebook(parameters_list):
        print(d, D, param)
        # we need try-except because on some combinations model fails to converge
        try:
            model=sm.tsa.statespace.SARIMAX(df['y'][:-predictions], order=(param[0], d, param[1]), 
                                            seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
        except:
            continue
        aic = model.aic
        # saving best model, AIC and parameters
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        results.append([D,d,  param, model.aic])

    result_table = pd.DataFrame(results)
    result_table.columns = ['D','d', 'parameters', 'aic']
    # sorting in ascending order, the lower AIC is - the better
    result_table = result_table.sort_values(by='aic', ascending=True).reset_index(drop=True)
    
    return result_table

In [15]:
%%time
result_table = pd.DataFrame()
for d in range(0,2):
    for D in range(0,3):
        result_table = pd.concat([result_table, optimizeSARIMA(parameters_list, d, D, s)])
        

0 0 (0, 1, 1, 1)


D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



0 0 (0, 1, 1, 2)


D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



0 0 (0, 1, 3, 1)


...

D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



1 2 (1, 2, 3, 1)


D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



1 2 (1, 2, 3, 2)

Wall time: 1min 44s


D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [16]:
# set the parameters that give the lowest AIC
result_table = result_table.reset_index().sort_values(['aic']).drop('index', 1)
Q, q, P,p = result_table.parameters[0]
d = result_table.d[0]
D = result_table.D[0]

best_model=sm.tsa.statespace.SARIMAX(df['y'][:-predictions], order=(p, d, q), 
                                        seasonal_order=(P, D, Q, s)).fit(disp=-1)
print(best_model.summary())

D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.

D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:578: ValueWarning:

An unsupported index was provided and will be ignored when e.g. forecasting.



                                     SARIMAX Results                                      
Dep. Variable:                                  y   No. Observations:                  353
Model:             SARIMAX(1, 0, 1)x(3, 0, [], 7)   Log Likelihood               -2577.044
Date:                            Tue, 19 Jan 2021   AIC                           5166.088
Time:                                    12:59:01   BIC                           5189.287
Sample:                                         0   HQIC                          5175.319
                                            - 353                                         
Covariance Type:                              opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.7188      0.080      8.971      0.000       0.562       0.876
ma.L1         -0.3119      0.096   

D:\Users\Eskander\Anaconda3\lib\site-packages\statsmodels\base\model.py:566: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals



In [17]:
D,d, Q, q, P,p

(0, 0, 0, 1, 3, 1)

In [18]:
result_table.head(n=18) #  в нашем случае это строка под индексом 85

,D,d,parameters,aic
0,0,0,"(0, 1, 3, 1)",12.000000
1,0,0,"(0, 1, 3, 2)",14.000000
48,0,1,"(1, 1, 3, 1)",14.000000
32,2,0,"(1, 2, 3, 1)",16.000000
2,0,0,"(1, 2, 3, 2)",75.321541
49,0,1,"(0, 1, 3, 2)",148.685262
80,2,1,"(0, 2, 3, 1)",4961.632629
81,2,1,"(1, 1, 3, 1)",4962.856541
82,2,1,"(0, 2, 1, 2)",4976.670061
33,2,0,"(0, 2, 3, 1)",4980.827422


**Next, we turn to the construction of the SARIMAX model (`sm.tsa.statespace.SARIMAX`).<br> <font color='red'>Question 5:</font> What parameters are the best for the model according to the `AIC` criterion?**

- D = 1, d = 0, Q = 0, q = 2, P = 3, p = 1
- D = 2, d = 1, Q = 1, q = 2, P = 3, p = 1 --
- D = 1, d = 1, Q = 1, q = 2, P = 3, p = 1
- D = 0, d = 0, Q = 0, q = 2, P = 3, p = 1